In [19]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import pickle
import csv
import numpy as np
from scipy.misc import imread
import shutil
import matplotlib.pyplot as plt

import h5py
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from utilities import *
# from Dataloader import *
# from read_images import *
train_dir = "data/model_train"
test_dir = "data/model_test"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
num_classes = 10
create_small_case(sel_whales = np.arange(1,num_classes+1),          # whales to be considered
                  small_dir = train_dir) 

old directory removed data/model_train
copy 34 images for whale # 1 in ordered list, called w_1287fbc
copy 27 images for whale # 2 in ordered list, called w_98baff9
copy 26 images for whale # 3 in ordered list, called w_7554f44
copy 23 images for whale # 4 in ordered list, called w_1eafe46
copy 22 images for whale # 5 in ordered list, called w_693c9ee
copy 22 images for whale # 6 in ordered list, called w_ab4cae2
copy 22 images for whale # 7 in ordered list, called w_fd1cb9d
copy 21 images for whale # 8 in ordered list, called w_43be268
copy 21 images for whale # 9 in ordered list, called w_73d5489
copy 21 images for whale # 10 in ordered list, called w_987a36f


In [23]:
# for first testing make test directory same as train directory (no real solution !!)
create_small_case(sel_whales = np.arange(1,num_classes+1),          # whales to be considered
                  small_dir = test_dir)

old directory removed data/model_test
copy 34 images for whale # 1 in ordered list, called w_1287fbc
copy 27 images for whale # 2 in ordered list, called w_98baff9
copy 26 images for whale # 3 in ordered list, called w_7554f44
copy 23 images for whale # 4 in ordered list, called w_1eafe46
copy 22 images for whale # 5 in ordered list, called w_693c9ee
copy 22 images for whale # 6 in ordered list, called w_ab4cae2
copy 22 images for whale # 7 in ordered list, called w_fd1cb9d
copy 21 images for whale # 8 in ordered list, called w_43be268
copy 21 images for whale # 9 in ordered list, called w_73d5489
copy 21 images for whale # 10 in ordered list, called w_987a36f


In [30]:
# from convertFilestructure import *

# generate image batches, replace by Keras ImageDataGenerator
# def input_generator(batch_size=64, directory="./data/test"):
#     i = 0
#     images = []
#     for filename in os.listdir(directory):
#         if i == batch_size:
#             i = 0
#             yield np.array(images)
#             images = []
#         images.append(skimage.io.imread(os.path.join(directory, filename)))
#         i += 1
        

# Use pretrained model as described in https://keras.io/applications/


def make_label_dict(directory=train_dir):
    label_dict = dict()
    for i, label in enumerate(sorted(os.listdir(directory))):
        label_dict[i] = label
    print("made label dict")
    return label_dict


# def main():
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
# num_classes = 300
# num_classes = 4251
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# define image generator
train_gen = image.ImageDataGenerator()

# train the model on the new data for a few epochs
model.fit_generator(train_gen.flow_from_directory(train_dir), verbose = 2,
                    steps_per_epoch=None, epochs=5)                        

# let's predict the test set to see a rough score
labels = make_label_dict(directory=train_dir)

    # at this point, the top layers are well trained and we can start fine-tuning
    # convolutional layers from inception V3. We will freeze the bottom N layers
    # and train the remaining top layers.

    # let's visualize layer names and layer indices to see how many layers
    # we should freeze:
    #for i, layer in enumerate(base_model.layers):
    #   print(i, layer.name)

    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 249 layers and unfreeze the rest:
    #for layer in model.layers[:249]:
    #   layer.trainable = False
    #for layer in model.layers[249:]:
    #   layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    #from keras.optimizers import SGD
    #model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

    # we train our model again (this time fine-tuning the top 2 inception blocks
    # alongside the top Dense layers
    #model.fit_generator(...)




Found 239 images belonging to 10 classes.
Epoch 1/5
 - 138s - loss: 9.7552
Epoch 2/5
 - 144s - loss: 2.9320
Epoch 3/5
 - 135s - loss: 2.0652
Epoch 4/5
 - 130s - loss: 1.6650
Epoch 5/5
 - 121s - loss: 2.0017
made label dict


In [31]:
test_gen = image.ImageDataGenerator()
preds = model.predict_generator(test_gen.flow_from_directory\
                                (test_dir, class_mode=None), verbose = 1, steps=None)
# decode the results into a list of tuples (class, description, probability)

Found 239 images belonging to 10 classes.
8/8 [==============================] - 171s 21s/step


In [39]:
print(preds.shape)
print(preds[:5])

(239, 10)
[[  1.38477849e-27   0.00000000e+00   0.00000000e+00   0.00000000e+00
    5.73760339e-19   0.00000000e+00   1.75125950e-26   1.00000000e+00
    1.29879905e-23   3.15279697e-20]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.14645392e-22   0.00000000e+00   7.07003153e-28   1.00000000e+00
    4.10193619e-26   2.65802090e-21]
 [  5.62011409e-24   0.00000000e+00   0.00000000e+00   0.00000000e+00
    4.04557411e-18   0.00000000e+00   5.73383425e-21   1.00000000e+00
    1.81090373e-23   9.67122925e-19]
 [  2.17668931e-38   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.65583387e-19   0.00000000e+00   9.25755463e-32   1.00000000e+00
    1.15018385e-25   1.00147760e-22]
 [  4.02923917e-30   0.00000000e+00   0.00000000e+00   0.00000000e+00
    5.69232029e-20   0.00000000e+00   5.17731652e-23   1.00000000e+00
    4.39664329e-25   1.92066996e-18]]


In [38]:
top_k = preds.argsort()[:, -4:][:, ::-1]
classes = [" ".join([labels[i] for i in line]) for line in top_k]
print(top_k.shape)
print(classes)

# filenames = test_gen.filenames
# csv_list = zip(filenames, classes)
# write_csv(csv_list, file_name="submission.csv")

(239, 4)
['w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_fd1cb9d w_73d5489 w_ab4cae2', 'w_98baff9 w_73d5489 w_fd1cb9d w_987a36f', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_fd1cb9d w_73d5489 w_987a36f', 'w_98baff9 w_fd1cb9d w_73d5489 w_ab4cae2', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_fd1cb9d w_ab4cae2 w_73d5489', 'w_98baff9 w_fd1cb9d w_73d5489 w_987a36f', 'w_98baff9 w_fd1cb9d w_987a36f w_73d5489', 'w_98baff9 w_fd1cb9d w_73d5489 w_ab4cae2', 'w_98baff9 w_fd1cb9d w_73d5489 w_987a36f', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_fd1cb9d w_73d5489 w_ab4cae2', 'w_98baff9 w_fd1cb9d w_987a36f w_ab4cae2', 'w_98baff9 w_73d5489 w_fd1cb9d w_987a36f', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_fd1cb9d w_73d5489 w_987a36f', 'w_98baff9 w_fd1cb9d w_ab4cae2 w_73d5489', 'w_98baff9 w_fd1cb9d w_73d5489 w_ab4cae2', 'w_98baff9 w_73d5489 w_987a36f w_fd1cb9d', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', 'w_98baff9 w_73d5489 w_fd1cb9d w_ab4cae2', '